In [1]:
import json

# Define a function to load JSON data from a file
def loadJson(filename):
    # Open the specified file in read mode, specifying utf-8 encoding
    with open(filename, 'r', encoding='utf-8') as file:
        # Load the JSON data from the file into a Python dictionary
        data = json.load(file)
    # Return the loaded data
    return data

In [ ]:
def writeJsonToFile(data, filePath, indent=4):
    with open(filePath, "w", encoding="utf-8") as jsonFile:
        json.dump(data, jsonFile, ensure_ascii=False, indent=indent)

    print("JSON data written to", filePath)

In [ ]:
import openai

def gptTranslateInfo(systemContent, inputInfo): 
    openai.api_base = 'https://pta-nbg-poc1.openai.azure.com/'
    openai.api_key = '89423897cfd94a89838586e836d26690'
    deployment_name = 'got-35-turbo1'
    openai.api_type = 'azure'
    openai.api_version = '2023-03-15-preview' # this may change in the future

    # systemContent = '''I will give you english column names from tables in english and you will translate it in greek. 
    # Please return only the translated column.'''

    # inputInfo = '''Translate the column "assets in million" to greek to fit the context of the table:
    # Table: company
    # Columns: company id, name, headquarters, industry, sales in million, assets in million'''
    try:
        response = openai.ChatCompletion.create(
            engine=deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
            temperature=0.3,
            messages=[
                {"role": "system", "content": systemContent},
                {"role": "user", "content": inputInfo}
            ]
        )
        return response['choices'][0]['message']['content']
    except KeyError:
        return ""

In [ ]:
import re

def removeTextInParentheses(inputString):
    # Define a regex pattern to match text within parentheses
    pattern = r'\([^)]*\)'
    
    # Use re.sub() to replace the matched pattern with an empty string
    outputString = re.sub(pattern, '', inputString)
    
    return outputString

In [ ]:
def createTableDict(database, tableNameAttribute = 'table_names_translated'):
    dictWithTables = {}
    # Extract table names from the database schema (assuming they are stored in 'table_names')
    tablesNames = database[tableNameAttribute]
    
    # Iterate over the table names and assign each a unique index as the key in the dictionary
    for index, tableName in enumerate(tablesNames):
        dictWithTables[index] = tableName
    
    return dictWithTables

In [ ]:
def returnTableAndColumnInfoForDbId(dictWithTablesData, dbId, columnNameAttribute = 'column_names_translated'):
    tableNameAttribute = ''
    if 'translated' in columnNameAttribute:
        tableNameAttribute = 'table_names_translated'
    else:
        tableNameAttribute = 'table_names'

    # Retrieve the database schema corresponding to the given dbId
    database = dictWithTablesData[dbId]
    # Create a dictionary of tables with their indices as keys
    dictWithTables = createTableDict(database, tableNameAttribute)

    # Initialize a list to store table and column information
    listWithTableAndColumnInfo = []

    # Iterate over the column information in the database schema
    for index, columnName in database[columnNameAttribute]:
        # Check if the column index is valid (-1 indicates unknown)
        if index != -1:
            # Retrieve the table name corresponding to the index from the dictionary
            tableName = dictWithTables[index]
            # Append the table name and column name to the list
            listWithTableAndColumnInfo.append([tableName, columnName])

    return listWithTableAndColumnInfo

In [ ]:
def createSchemaInfoDict(listWithTableAndColumnInfo):
    # Create an empty dictionary to store table names as keys and corresponding column names as values.
    dictWithSchemaInfo = {}

    # Iterate through pairs of table names and column names.
    for tableName, columnName in listWithTableAndColumnInfo:
        # Check if both the table name and column name are not special values.
        if tableName != -1 and columnName != '*':
            # Check if the table name is not already in the dictionary.
            if tableName not in dictWithSchemaInfo.keys():
                # If not, initialize the entry with a list containing the current column name.
                dictWithSchemaInfo[tableName] = [columnName]
            else:
                # If the table name is already in the dictionary, append the current column name to its list of column names.
                dictWithSchemaInfo[tableName].append(columnName)
    return dictWithSchemaInfo

In [ ]:
def createTheDesiredPromtForQuestionsTranslation(question, dictWithSchemaInfo, language='Greek'):
    infoString = ""

    for tableName, columnNames in dictWithSchemaInfo.items():
        infoString += f"Table: {tableName}\n"
        infoString += f"Columns: {', '.join(columnNames)}\n"
    
    return f'''Translate the question "{question}" to {language}. You can only use keywords provided below:\n{infoString}'''

In [ ]:
import nltk

def tokenizeSentence(sentence):
    # Tokenize the input sentence using nltk.word_tokenize()
    tokens = nltk.word_tokenize(sentence)
    return tokens